In [542]:
#!pip install xgboost

In [543]:
#!pip install catboost

In [544]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier

train = pd.read_csv('./data/train.csv', index_col='no')
test = pd.read_csv('./data/test.csv', index_col='no')

def strip_title(row):
    return row.strip()

# train 공백제거
# except pass 는 수치형나오면 오류나는거를 pass한것
for i in range(len(train.columns)):
    try:
        train.iloc[:,i] = train.iloc[:,i].apply(strip_title)
    except:
        pass
# test 공백제거
for i in range(len(test.columns)):
    try:
        test.iloc[:,i] = test.iloc[:,i].apply(strip_title)
    except:
        pass


# ?을 other로 변환
train['occupation'] = train['occupation'].replace('?','other')
train['workclass'] = train['workclass'].replace('?','other')
train['native-country'] = train['native-country'].replace('?','other')

# ?을 other로 변환
test['occupation'] = test['occupation'].replace('?','other')
test['workclass'] = test['workclass'].replace('?','other')
test['native-country'] = test['native-country'].replace('?','other')

# train.drop(['fnlwgt','capital-gain', 'capital-loss','education'],axis=1,inplace=True)
# test.drop(['fnlwgt','capital-gain', 'capital-loss','education'],axis=1,inplace=True)

train.drop(['fnlwgt','education'],axis=1,inplace=True)
test.drop(['fnlwgt','education'],axis=1,inplace=True)

train['native-country'] = train['native-country'].map({'United-States':'United-States',"other":'United-States'})
train['native-country'].fillna("other-country",inplace=True)
test['native-country'] = test['native-country'].map({'United-States':'United-States',"other":'United-States'})
test['native-country'].fillna("other-country",inplace=True)


In [545]:
# 인코딩할 컬럼명만 선택
categorical_features = ['workclass', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']
# categorical_features = ['workclass','occupation']
# 반복문으로 인코딩하기
for feature_name in categorical_features:
    one_hot = pd.get_dummies(train[feature_name], prefix = feature_name)
    train = pd.concat([train,one_hot], axis=1) # 기존 데이터 끝에 one_hot을 붙임.
    train.drop(feature_name, axis=1, inplace=True) # 기존 글자컬럼을 삭제
    
for feature_name in categorical_features:
    one_hot = pd.get_dummies(test[feature_name], prefix = feature_name)
    test = pd.concat([test,one_hot], axis=1) # 기존 데이터 끝에 one_hot을 붙임.
    test.drop(feature_name, axis=1, inplace=True) # 기존 글자컬럼을 삭제

In [546]:
train.shape

(29305, 52)

In [547]:
test.shape

(19537, 51)

In [548]:
train.drop("workclass_other", axis = 1, inplace = True)
test.drop("workclass_other", axis = 1, inplace = True)

In [549]:

train.sort_index(axis=1, inplace=True)
test.sort_index(axis=1, inplace=True)
X_train = train.drop(['income'], axis=1)
y_train = train.income
X_test=test
X_train.sort_index(axis=1, inplace=True)
X_test.sort_index(axis=1, inplace=True)


In [550]:
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, random_state=0)

In [551]:
# xgb_model = XGBClassifier(booster='gbtree', # 모델의 종류 (이 값은 고정)
#                           n_estimators=500, # 앙상블 모델 갯수
#                           learning_rate=0.1, # 학습률
#                           max_depth = 3, # 최대깊이(일반적으로 깊게 설정x)
#                           gamma = 0.2, # 리프노드를 어떻게 나눌것이냐(값이 크면 모델 단순, 작아지면 모델이 복잡) 
#                           random_state=721
#                          )
# xgb = cross_val_score(xgb_model,X_train,y_train,cv=5)
# print (xgb.mean())

In [552]:
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, random_state=0)
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score

catboost = CatBoostClassifier(iterations=3000, random_state=0)

catboost.fit(X_train, y_train)
pred_cat = catboost.predict(X_test)


Learning rate set to 0.012448
0:	learn: 0.6811012	total: 3.44ms	remaining: 10.3s
1:	learn: 0.6682044	total: 6.72ms	remaining: 10.1s
2:	learn: 0.6572096	total: 10ms	remaining: 10s
3:	learn: 0.6448738	total: 13.5ms	remaining: 10.1s
4:	learn: 0.6343894	total: 16.8ms	remaining: 10s
5:	learn: 0.6248340	total: 20.1ms	remaining: 10s
6:	learn: 0.6159034	total: 22.3ms	remaining: 9.53s
7:	learn: 0.6062416	total: 25.6ms	remaining: 9.58s
8:	learn: 0.5969795	total: 29.2ms	remaining: 9.7s
9:	learn: 0.5872344	total: 32.5ms	remaining: 9.7s
10:	learn: 0.5793417	total: 35.7ms	remaining: 9.69s
11:	learn: 0.5711498	total: 38.8ms	remaining: 9.66s
12:	learn: 0.5624539	total: 41.9ms	remaining: 9.63s
13:	learn: 0.5551556	total: 45.2ms	remaining: 9.65s
14:	learn: 0.5477939	total: 48.6ms	remaining: 9.67s
15:	learn: 0.5399647	total: 51.9ms	remaining: 9.67s
16:	learn: 0.5335815	total: 55.1ms	remaining: 9.66s
17:	learn: 0.5273097	total: 58.4ms	remaining: 9.67s
18:	learn: 0.5206490	total: 61.8ms	remaining: 9.7s
19:

164:	learn: 0.3193956	total: 547ms	remaining: 9.4s
165:	learn: 0.3191337	total: 550ms	remaining: 9.4s
166:	learn: 0.3188547	total: 554ms	remaining: 9.39s
167:	learn: 0.3186203	total: 557ms	remaining: 9.39s
168:	learn: 0.3183499	total: 560ms	remaining: 9.38s
169:	learn: 0.3182025	total: 564ms	remaining: 9.38s
170:	learn: 0.3180018	total: 567ms	remaining: 9.38s
171:	learn: 0.3178087	total: 570ms	remaining: 9.37s
172:	learn: 0.3176660	total: 573ms	remaining: 9.37s
173:	learn: 0.3174902	total: 577ms	remaining: 9.37s
174:	learn: 0.3172755	total: 580ms	remaining: 9.37s
175:	learn: 0.3171094	total: 583ms	remaining: 9.36s
176:	learn: 0.3168650	total: 587ms	remaining: 9.36s
177:	learn: 0.3166465	total: 590ms	remaining: 9.36s
178:	learn: 0.3164529	total: 593ms	remaining: 9.35s
179:	learn: 0.3162576	total: 597ms	remaining: 9.35s
180:	learn: 0.3160923	total: 600ms	remaining: 9.35s
181:	learn: 0.3158952	total: 603ms	remaining: 9.34s
182:	learn: 0.3157302	total: 607ms	remaining: 9.34s
183:	learn: 0.

329:	learn: 0.2984531	total: 1.09s	remaining: 8.84s
330:	learn: 0.2983904	total: 1.09s	remaining: 8.84s
331:	learn: 0.2982186	total: 1.1s	remaining: 8.83s
332:	learn: 0.2981434	total: 1.1s	remaining: 8.83s
333:	learn: 0.2980735	total: 1.1s	remaining: 8.83s
334:	learn: 0.2980099	total: 1.11s	remaining: 8.82s
335:	learn: 0.2979444	total: 1.11s	remaining: 8.82s
336:	learn: 0.2978086	total: 1.11s	remaining: 8.81s
337:	learn: 0.2977341	total: 1.12s	remaining: 8.81s
338:	learn: 0.2976761	total: 1.12s	remaining: 8.81s
339:	learn: 0.2975581	total: 1.13s	remaining: 8.8s
340:	learn: 0.2974859	total: 1.13s	remaining: 8.8s
341:	learn: 0.2974164	total: 1.13s	remaining: 8.8s
342:	learn: 0.2973307	total: 1.14s	remaining: 8.79s
343:	learn: 0.2972596	total: 1.14s	remaining: 8.79s
344:	learn: 0.2971440	total: 1.14s	remaining: 8.79s
345:	learn: 0.2970969	total: 1.15s	remaining: 8.79s
346:	learn: 0.2970229	total: 1.15s	remaining: 8.78s
347:	learn: 0.2969761	total: 1.15s	remaining: 8.78s
348:	learn: 0.2968

496:	learn: 0.2877732	total: 1.64s	remaining: 8.26s
497:	learn: 0.2877298	total: 1.64s	remaining: 8.26s
498:	learn: 0.2876754	total: 1.65s	remaining: 8.25s
499:	learn: 0.2876546	total: 1.65s	remaining: 8.25s
500:	learn: 0.2875967	total: 1.65s	remaining: 8.24s
501:	learn: 0.2875668	total: 1.66s	remaining: 8.24s
502:	learn: 0.2875185	total: 1.66s	remaining: 8.24s
503:	learn: 0.2874822	total: 1.66s	remaining: 8.23s
504:	learn: 0.2874294	total: 1.67s	remaining: 8.23s
505:	learn: 0.2873757	total: 1.67s	remaining: 8.22s
506:	learn: 0.2873372	total: 1.67s	remaining: 8.22s
507:	learn: 0.2873012	total: 1.68s	remaining: 8.22s
508:	learn: 0.2872716	total: 1.68s	remaining: 8.21s
509:	learn: 0.2872131	total: 1.68s	remaining: 8.21s
510:	learn: 0.2871776	total: 1.68s	remaining: 8.21s
511:	learn: 0.2871414	total: 1.69s	remaining: 8.2s
512:	learn: 0.2870971	total: 1.69s	remaining: 8.2s
513:	learn: 0.2870332	total: 1.69s	remaining: 8.2s
514:	learn: 0.2869941	total: 1.7s	remaining: 8.19s
515:	learn: 0.28

664:	learn: 0.2807145	total: 2.19s	remaining: 7.68s
665:	learn: 0.2806594	total: 2.19s	remaining: 7.68s
666:	learn: 0.2806226	total: 2.19s	remaining: 7.67s
667:	learn: 0.2805960	total: 2.2s	remaining: 7.67s
668:	learn: 0.2805597	total: 2.2s	remaining: 7.67s
669:	learn: 0.2804717	total: 2.2s	remaining: 7.67s
670:	learn: 0.2804470	total: 2.21s	remaining: 7.66s
671:	learn: 0.2804193	total: 2.21s	remaining: 7.66s
672:	learn: 0.2803783	total: 2.21s	remaining: 7.65s
673:	learn: 0.2802997	total: 2.22s	remaining: 7.65s
674:	learn: 0.2802668	total: 2.22s	remaining: 7.65s
675:	learn: 0.2802270	total: 2.22s	remaining: 7.64s
676:	learn: 0.2801874	total: 2.23s	remaining: 7.64s
677:	learn: 0.2801615	total: 2.23s	remaining: 7.64s
678:	learn: 0.2801253	total: 2.23s	remaining: 7.63s
679:	learn: 0.2800928	total: 2.24s	remaining: 7.63s
680:	learn: 0.2800563	total: 2.24s	remaining: 7.63s
681:	learn: 0.2800157	total: 2.24s	remaining: 7.62s
682:	learn: 0.2799872	total: 2.25s	remaining: 7.62s
683:	learn: 0.2

...

2329:	learn: 0.2404344	total: 7.66s	remaining: 2.2s
2330:	learn: 0.2404178	total: 7.66s	remaining: 2.2s
2331:	learn: 0.2403955	total: 7.67s	remaining: 2.2s
2332:	learn: 0.2403828	total: 7.67s	remaining: 2.19s
2333:	learn: 0.2403712	total: 7.67s	remaining: 2.19s
2334:	learn: 0.2403527	total: 7.68s	remaining: 2.19s
2335:	learn: 0.2403360	total: 7.68s	remaining: 2.18s
2336:	learn: 0.2403299	total: 7.68s	remaining: 2.18s
2337:	learn: 0.2403134	total: 7.69s	remaining: 2.18s
2338:	learn: 0.2403052	total: 7.69s	remaining: 2.17s
2339:	learn: 0.2402833	total: 7.69s	remaining: 2.17s
2340:	learn: 0.2402682	total: 7.7s	remaining: 2.17s
2341:	learn: 0.2402576	total: 7.7s	remaining: 2.16s
2342:	learn: 0.2402423	total: 7.7s	remaining: 2.16s
2343:	learn: 0.2402313	total: 7.71s	remaining: 2.16s
2344:	learn: 0.2402150	total: 7.71s	remaining: 2.15s
2345:	learn: 0.2402053	total: 7.71s	remaining: 2.15s
2346:	learn: 0.2401974	total: 7.72s	remaining: 2.15s
2347:	learn: 0.2401829	total: 7.72s	remaining: 2.14s

2494:	learn: 0.2380411	total: 8.2s	remaining: 1.66s
2495:	learn: 0.2380210	total: 8.21s	remaining: 1.66s
2496:	learn: 0.2380029	total: 8.21s	remaining: 1.65s
2497:	learn: 0.2379802	total: 8.21s	remaining: 1.65s
2498:	learn: 0.2379622	total: 8.22s	remaining: 1.65s
2499:	learn: 0.2379506	total: 8.22s	remaining: 1.64s
2500:	learn: 0.2379388	total: 8.22s	remaining: 1.64s
2501:	learn: 0.2379238	total: 8.23s	remaining: 1.64s
2502:	learn: 0.2379080	total: 8.23s	remaining: 1.63s
2503:	learn: 0.2378995	total: 8.23s	remaining: 1.63s
2504:	learn: 0.2378912	total: 8.24s	remaining: 1.63s
2505:	learn: 0.2378737	total: 8.24s	remaining: 1.62s
2506:	learn: 0.2378587	total: 8.24s	remaining: 1.62s
2507:	learn: 0.2378452	total: 8.25s	remaining: 1.62s
2508:	learn: 0.2378281	total: 8.25s	remaining: 1.61s
2509:	learn: 0.2378133	total: 8.25s	remaining: 1.61s
2510:	learn: 0.2377977	total: 8.26s	remaining: 1.61s
2511:	learn: 0.2377783	total: 8.26s	remaining: 1.6s
2512:	learn: 0.2377681	total: 8.26s	remaining: 1

2658:	learn: 0.2357403	total: 8.75s	remaining: 1.12s
2659:	learn: 0.2357233	total: 8.75s	remaining: 1.12s
2660:	learn: 0.2357154	total: 8.76s	remaining: 1.11s
2661:	learn: 0.2357069	total: 8.76s	remaining: 1.11s
2662:	learn: 0.2356860	total: 8.77s	remaining: 1.11s
2663:	learn: 0.2356784	total: 8.77s	remaining: 1.11s
2664:	learn: 0.2356580	total: 8.77s	remaining: 1.1s
2665:	learn: 0.2356446	total: 8.78s	remaining: 1.1s
2666:	learn: 0.2356043	total: 8.78s	remaining: 1.1s
2667:	learn: 0.2355941	total: 8.78s	remaining: 1.09s
2668:	learn: 0.2355664	total: 8.79s	remaining: 1.09s
2669:	learn: 0.2355455	total: 8.79s	remaining: 1.09s
2670:	learn: 0.2355144	total: 8.79s	remaining: 1.08s
2671:	learn: 0.2355049	total: 8.8s	remaining: 1.08s
2672:	learn: 0.2354806	total: 8.8s	remaining: 1.08s
2673:	learn: 0.2354713	total: 8.8s	remaining: 1.07s
2674:	learn: 0.2354547	total: 8.81s	remaining: 1.07s
2675:	learn: 0.2354399	total: 8.81s	remaining: 1.07s
2676:	learn: 0.2354300	total: 8.81s	remaining: 1.06s

2825:	learn: 0.2332987	total: 9.3s	remaining: 572ms
2826:	learn: 0.2332922	total: 9.3s	remaining: 569ms
2827:	learn: 0.2332746	total: 9.3s	remaining: 566ms
2828:	learn: 0.2332581	total: 9.31s	remaining: 563ms
2829:	learn: 0.2332315	total: 9.31s	remaining: 559ms
2830:	learn: 0.2332195	total: 9.31s	remaining: 556ms
2831:	learn: 0.2332066	total: 9.32s	remaining: 553ms
2832:	learn: 0.2331905	total: 9.32s	remaining: 549ms
2833:	learn: 0.2331831	total: 9.32s	remaining: 546ms
2834:	learn: 0.2331706	total: 9.33s	remaining: 543ms
2835:	learn: 0.2331634	total: 9.33s	remaining: 540ms
2836:	learn: 0.2331536	total: 9.33s	remaining: 536ms
2837:	learn: 0.2331331	total: 9.34s	remaining: 533ms
2838:	learn: 0.2331268	total: 9.34s	remaining: 530ms
2839:	learn: 0.2331206	total: 9.34s	remaining: 526ms
2840:	learn: 0.2330979	total: 9.35s	remaining: 523ms
2841:	learn: 0.2330864	total: 9.35s	remaining: 520ms
2842:	learn: 0.2330760	total: 9.35s	remaining: 517ms
2843:	learn: 0.2330594	total: 9.36s	remaining: 51

2994:	learn: 0.2309916	total: 9.84s	remaining: 16.4ms
2995:	learn: 0.2309656	total: 9.85s	remaining: 13.1ms
2996:	learn: 0.2309534	total: 9.85s	remaining: 9.86ms
2997:	learn: 0.2309362	total: 9.85s	remaining: 6.57ms
2998:	learn: 0.2309300	total: 9.86s	remaining: 3.29ms
2999:	learn: 0.2309251	total: 9.86s	remaining: 0us


In [553]:
print(accuracy_score(pred_cat, y_test))

0.8666060054595086


In [554]:
# 5000 - 0.8748464583048997 random_state = 0
#print(accuracy_score(pred_cat, y_test))

In [555]:
# 3000 - 0.8747099767981439
#print(accuracy_score(pred_cat, y_test))

In [556]:
# 4000
#print(accuracy_score(pred_cat, y_test))

In [557]:
# 3000 - 0.8747099767981439 - random_state =10
#print(accuracy_score(pred_cat, y_test))

In [558]:
# 355 - 0.8877551020408163
#print(accuracy_score(pred_cat, y_test))

In [559]:

#print(accuracy_score(pred_cat, y_test))

In [560]:
# 모델 random_state = 712 0.9230769230769231
#print(accuracy_score(pred_cat, y_test))

In [561]:
# 모델 random_state = 355,712 355,791     0.86787
#print(accuracy_score(pred_cat, y_test))

In [562]:
# 10, iterations=3550, random_state=519 0.8731574158325751

In [563]:
# 10, iterations=355, random_state=0 0.8740673339399454

In [564]:
# 10, iterations=335, random_state=0

In [565]:
# bg2 는 덮어씌워서 없어짐

# pre = catboost.predict(test)
# submission = pd.read_csv("./data/sample_submission.csv",index_col="no")
# submission["income"] = pre
# submission.to_csv('data1/bg3.csv')
